<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/logo.png" width="128"/>
<img align="right" src="images/etcbc.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

previous: [start](start.ipynb)

---

# Sharing data features

In [ ]:
import collections
import os

from tf.app import use

In [ ]:
A = use('dss', hoist=globals(), silent='deep')
silentOff()

# Making data

A certain consonant gets `cert=100`.

Penalties:

feature | value | penalty | description
--- | --- | --- | ---
unc | 1 | 10 | uncertainty level 1
unc | 2 | 20 | uncertainty level 2
unc | 3 | 30 | uncertainty level 3
unc | 4 | 40 | uncertainty level 4
rec | 1 | 45 | modern reconstruction
rem | 1 | 40 | modern removal
rem | 2 | 20 | ancient removal
cor | 1 | 18 | modern correction
cor | 2 | 12 | ancient correction
alt | 1 | 25 | alternate reading

The minimum is `1`.

We extend the `cert` measure to words, fragments and scrolls by averaging.

## The measure function

In [ ]:
def measure(s):
  c = 100
  d = F.unc.v(s)
  if d:
    c -= 10 * d
  d = F.rec.v(s)
  if d:
    c -= 45
  d = F.rem.v(s)
  if d == 1:
    c -= 40
  elif d == 2:
    c -= 20
  d = F.cor.v(s)
  if d == 2 or d == 3:
    c -= 12
  elif d == 1:
    c -= 18
  d = F.alt.v(s)
  if d:
    c -= 25
  if c < 1:
    c = 1
  return c

## Measure everything

In [ ]:
CONS = 'cons'
cert = {}

indent(reset=True)

for sc in F.otype.s('scroll'):
  fN = 0
  fSum = 0
  
  for f in L.d(sc, otype='fragment'):
    lN = 0
    lSum = 0
    
    for l in L.d(f, otype='line'):
      wN = 0
      wSum = 0
      
      for w in L.d(l, otype='word'):
        sN = 0
        sSum = 0
        
        for s in L.d(w, otype='sign'):
          if F.type.v(s) != CONS:
            continue
          sCert = measure(s)
          cert[s] = sCert
          sN += 1
          sSum += sCert
          
        if sN:
          wCert = int(round(sSum / sN))
          cert[w] = wCert
          wN += 1
          wSum += wCert
          
      if wN:
        lCert = int(round(wSum / wN))
        cert[l] = lCert
        lN += 1
        lSum += lCert
        
    if lN:
      fCert = int(round(lSum / lN))
      cert[f] = fCert
      fN += 1
      fSum += fCert
      
  if fN:
    scCert = int(round(fSum / fN))
    cert[sc] = scCert
    
info(f'{len(cert)} certainties determined')

# Saving data

## Location

In [ ]:
GITHUB = os.path.expanduser('~/github')
ORG = 'ancient-data'
REPO = 'dss'
PATH = 'exercises'
VERSION = A.version

## Metadata

In [ ]:
metaData = {
  'cert': dict(
    valueType='int',
    description='measure of certainty of material, between 1 and 100 (most certain)',
    creator='Dirk Roorda',
  ),
}

## Save action

In [ ]:
TF.save(nodeFeatures=dict(cert=cert), metaData=metaData, location=f'{GITHUB}/{ORG}/{REPO}/{PATH}/tf', module=VERSION)

# Share the data

In [ ]:
%%bash -s "$GITHUB" "$ORG" "$REPO" 
cd $1/$2/$3
git add --all .
git commit -m "cert feature added"
git push origin master

---

next: [use](use.ipynb)

---

full tutorial: [annotation/tutorials/dss](https://nbviewer.jupyter.org/github/annotation/tutorials/blob/master/dss/start.ipynb)